In [ ]:
import pandas as pd

adult_census = pd.read_csv("https://github.com/fenago/MLEssentials2/blob/main/datasets/adult-census.csv?raw=true")


In [ ]:
# to display nice model diagram
from sklearn import set_config
set_config(display='diagram')

In [ ]:
target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=target_name)

In [ ]:
numerical_columns = [
    "age", "capital-gain", "capital-loss", "hours-per-week"]

data_numeric = data[numerical_columns]

In [ ]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data_numeric, target, random_state=42)

In [ ]:
data_train.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(data_train)

In [ ]:
scaler.mean_

In [ ]:
scaler.scale_

In [ ]:
data_train_scaled = scaler.transform(data_train)
data_train_scaled

In [ ]:
data_train_scaled = scaler.fit_transform(data_train)
data_train_scaled

In [ ]:
data_train_scaled = pd.DataFrame(data_train_scaled,
                                 columns=data_train.columns)
data_train_scaled.describe()

In [ ]:
import matplotlib.pyplot  as plt
import seaborn as sns

# number of points to visualize to have a clearer plot
num_points_to_plot = 300

sns.jointplot(data=data_train[:num_points_to_plot], x="age",
              y="hours-per-week", marginal_kws=dict(bins=15))
plt.suptitle("Jointplot of 'age' vs 'hours-per-week' \nbefore StandardScaler", y=1.1)

sns.jointplot(data=data_train_scaled[:num_points_to_plot], x="age",
              y="hours-per-week", marginal_kws=dict(bins=15))
_ = plt.suptitle("Jointplot of 'age' vs 'hours-per-week' \nafter StandardScaler", y=1.1)

In [ ]:
import time
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(), LogisticRegression())
model

In [ ]:
model.named_steps

In [ ]:
start = time.time()
model.fit(data_train, target_train)
elapsed_time = time.time() - start

In [ ]:
predicted_target = model.predict(data_test)
predicted_target[:5]

In [ ]:
model_name = model.__class__.__name__
score = model.score(data_test, target_test)
print(f"The accuracy using a {model_name} is {score:.3f} "
      f"with a fitting time of {elapsed_time:.3f} seconds "
      f"in {model[-1].n_iter_[0]} iterations")

In [ ]:
model = LogisticRegression()
start = time.time()
model.fit(data_train, target_train)
elapsed_time = time.time() - start

In [ ]:
model_name = model.__class__.__name__
score = model.score(data_test, target_test)
print(f"The accuracy using a {model_name} is {score:.3f} "
      f"with a fitting time of {elapsed_time:.3f} seconds "
      f"in {model.n_iter_[0]} iterations")

In [ ]:
%%time
from sklearn.model_selection import cross_validate

model = make_pipeline(StandardScaler(), LogisticRegression())
cv_result = cross_validate(model, data_numeric, target, cv=5)
cv_result

In [ ]:
scores = cv_result["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} +/- {scores.std():.3f}"
)

In [ ]:
data["native-country"].value_counts().sort_index()

In [ ]:
data.dtypes

In [ ]:
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(data)
categorical_columns

In [ ]:
data_categorical = data[categorical_columns]
data_categorical.head()

In [ ]:
print(f"The dataset is composed of {data_categorical.shape[1]} features")

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

education_column = data_categorical[["education"]]

encoder = OrdinalEncoder()
education_encoded = encoder.fit_transform(education_column)
education_encoded

In [ ]:
encoder.categories_

In [ ]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

In [ ]:
print(f"The dataset encoded contains {data_encoded.shape[1]} features")

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
education_encoded = encoder.fit_transform(education_column)
education_encoded

In [ ]:
feature_names = encoder.get_feature_names_out(input_features=["education"])
education_encoded = pd.DataFrame(education_encoded, columns=feature_names)
education_encoded

In [ ]:
print(f"The dataset is composed of {data_categorical.shape[1]} features")
data_categorical.head()

In [ ]:
data_encoded = encoder.fit_transform(data_categorical)
data_encoded[:5]

In [ ]:
print(f"The encoded dataset contains {data_encoded.shape[1]} features")

In [ ]:
columns_encoded = encoder.get_feature_names_out(data_categorical.columns)
pd.DataFrame(data_encoded, columns=columns_encoded).head()

In [ ]:
data["native-country"].value_counts()

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

model = make_pipeline(
    OneHotEncoder(handle_unknown="ignore"), LogisticRegression(max_iter=500)
)

In [ ]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(model, data_categorical, target)
cv_results

In [ ]:
scores = cv_results["test_score"]
print(f"The accuracy is: {scores.mean():.3f} +/- {scores.std():.3f}")